<a href="https://colab.research.google.com/github/foxtrotmike/CS909/blob/master/autodiff_backprop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Building an Automatic Differentiation Library: Reverse-Mode (Backpropagation)**
(Fayyaz Minhas)


Automatic differentiation (**autodiff**) is a fundamental tool in modern machine learning and scientific computing.  

Unlike **forward-mode autodiff**, which propagates derivatives forward using **dual numbers**, **reverse-mode autodiff** computes derivatives **backward** using the **chain rule**.  

### **What You Will Learn**
1. **How reverse-mode autodiff works**
2. **Why reverse-mode is efficient for functions with many inputs**
3. **How to implement reverse-mode autodiff from scratch in Python**
4. **Computing gradients for**:

$$
e = (a + b)(b + 1)
$$

---


## **1️⃣ Understanding Reverse-Mode Autodiff**

### **🔹 Why Use Reverse-Mode?**  
Reverse-mode autodiff is useful for computing gradients **when a function has many inputs and one output**, such as **loss functions in deep learning**.  

For a function:  
$$
e = f(a, b)
$$
Reverse-mode computes **all gradients** in a **single backward pass**, making it **much more efficient than forward-mode** for large-scale optimization problems.

Historical note:

While, Forward-mode differentiation has roots in dual numbers (1873) and was formalized in Wengert’s work (1964), Seppo Linnainmaa (1970) is credited with the first formal description of reverse-mode autodiff in his PhD dissertation, which introduced the idea of storing intermediate results and computing gradients backward.



## **🔹 How Reverse-Mode Works (Computation Graph)**

Instead of using **dual numbers**, reverse-mode constructs a **computation graph** during the forward pass, then applies the **chain rule backward** to compute gradients efficiently.

For the function:

$$
e = (a + b)(b + 1)
$$

At $ a = 2 $, $ b = 3 $, we compute:

---

## **1️⃣ Forward Pass (Compute Function Value)**
To break the function into intermediate steps:

    a        b
     \      /  
      \    /
        c     d
         \   /
           e


### **Step 1: Compute $ c $**
$$
c = a + b
$$
Substituting $ a = 2 $, $ b = 3 $:
$$
c = 2 + 3 = 5
$$

### **Step 2: Compute $ d $**
$$
d = b + 1
$$
Substituting $ b = 3 $:
$$
d = 3 + 1 = 4
$$

### **Step 3: Compute $ e $**
$$
e = c \cdot d
$$
Substituting $ c = 5 $, $ d = 4 $:
$$
e = 5 \times 4 = 20
$$

At this point, we have computed the final output value $ e = 20 $. Now, we compute the gradients by applying the **chain rule step by step**.

---

## **2️⃣ Backward Pass (Applying Chain Rule)**
Now, we compute the **gradients of $ e $ with respect to $ a $ and $ b $ using the chain rule**.

We begin with derivative at the last step which is `e.grad`, i.e., derivative of the final value with respect to $e$:

$$
\frac{\partial e}{\partial e} = 1
$$
### **Step 1: Compute $ \frac{\partial e}{\partial c} $ and $ \frac{\partial e}{\partial d} $**
Since:
$$
e = c \cdot d
$$
Applying the derivative of a product gives us `c.grad`, i.e., derivative of the final value with respect to $c$ and `d.grad`, i.e., derivative of the final value with respect to $d$ as:
$$
\frac{\partial e}{\partial c} = d, \quad \frac{\partial e}{\partial d} = c
$$
Substituting $ c = 5 $, $ d = 4 $:
$$
\frac{\partial e}{\partial c} = 4, \quad \frac{\partial e}{\partial d} = 5
$$

### **Step 2: Compute $ \frac{\partial c}{\partial a} $ and $ \frac{\partial c}{\partial b} $**
Since:
$$
c = a + b
$$
Differentiating:
$$
\frac{\partial c}{\partial a} = 1, \quad \frac{\partial c}{\partial b} = 1
$$

### **Step 3: Compute $ \frac{\partial d}{\partial b} $**
Since:
$$
d = b + 1
$$
Differentiating:
$$
\frac{\partial d}{\partial b} = 1
$$

---

## **3️⃣ Compute Gradients Using Chain Rule**
Now, we apply the **chain rule** step by step.

### **Computing $ \frac{\partial e}{\partial a} $**

Using the chain rule:
$$
\frac{\partial e}{\partial a} = \frac{\partial e}{\partial c} \cdot \frac{\partial c}{\partial a}
$$
Substituting known values:
$$
\frac{\partial e}{\partial a} = 4 \times 1 = 4
$$

### **Computing $ \frac{\partial e}{\partial b} $**

Applying the chain rule for $ b $, since $ e $ depends on $ b $ through both $ c $ and $ d $:
$$
\frac{\partial e}{\partial b} = \frac{\partial e}{\partial c} \cdot \frac{\partial c}{\partial b} + \frac{\partial e}{\partial d} \cdot \frac{\partial d}{\partial b}
$$
Substituting known values:
$$
\frac{\partial e}{\partial b} = (4 \times 1) + (5 \times 1) = 4 + 5 = 9
$$

---

## **Final Gradients**
Thus, the computed gradients are:
$$
\frac{\partial e}{\partial a} = 4, \quad \frac{\partial e}{\partial b} = 9
$$

✅ **Reverse-mode autodiff efficiently computes all gradients in one backward pass!** 🚀


## **2️⃣ Implementing Reverse-Mode Autodiff in Python**
We will now implement reverse-mode autodiff step by step.
The `Tensor` class provides the foundation for reverse-mode automatic differentiation by **managing both values and their gradients** while dynamically building a computation graph. Instead of computing derivatives manually, this implementation **attaches a small function to each operation** that describes how its result depends on its inputs and how the gradients should propagate when differentiation is performed. Each operation, like addition or multiplication, registers **how it should compute its own derivative**, ensuring that when `.backward()` is called, the system can **traverse the computation graph in reverse**, applying the **chain rule** recursively. This allows complex functions to be differentiated automatically by **tracking dependencies** rather than symbolically solving for derivatives.

You are welcome to try and add other operators to it.



In [1]:
class DiffScalar:
    def __init__(self, value, requires_grad=False):
        """
        Initialize a scalar value that supports automatic differentiation.

        Arguments:
        - value: The numerical value of the scalar.
        - requires_grad: Whether this scalar requires gradient computation.

        Attributes:
        - grad: Stores the gradient of L (final output) wrt to this variable (set to 0 if differentiation is needed).
        - _backward: A function that defines how gradients should be propagated.
        - _prev: Tracks the dependent variables (parents in computation graph).
        """
        self.value = value
        self.requires_grad = requires_grad
        self.grad = 0 if requires_grad else None  # Initialize gradient
        self._backward = lambda: None  # Placeholder for gradient propagation
        self._prev = set()  # Store references to parent nodes in computation graph

    def __add__(self, other):
        """
        Define addition operation while maintaining the computation graph.

        If self and other are differentiable (requires_grad=True),
        then their gradients must be tracked for backpropagation.
        """
        other = other if isinstance(other, DiffScalar) else DiffScalar(other)  # Ensure other is a DiffScalar
        out = DiffScalar(self.value + other.value, requires_grad=self.requires_grad or other.requires_grad)

        def _backward():
            """
            Compute the gradient for self and other using the chain rule.

            Since d/dx (x + y) = 1, we propagate out.grad directly to both.

            We use += because the variable might be involved in multiple operations,
            so its gradient accumulates contributions from multiple sources.
            """
            if self.requires_grad:
                self.grad += out.grad  # ∂L/∂self += ∂L/∂out * ∂out/∂self (which is 1)
            if other.requires_grad:
                other.grad += out.grad  # ∂L/∂other += ∂L/∂out * ∂out/∂other (which is 1)

        out._backward = _backward  # Store the backward function for this operation
        out._prev = {self, other}  # Track dependencies
        return out

    def __mul__(self, other):
        """
        Define multiplication operation while maintaining the computation graph.

        If self and other are differentiable (requires_grad=True),
        then their gradients must be tracked for backpropagation.
        """
        other = other if isinstance(other, DiffScalar) else DiffScalar(other)  # Ensure other is a DiffScalar
        out = DiffScalar(self.value * other.value, requires_grad=self.requires_grad or other.requires_grad)

        def _backward():
            """
            Compute the gradient for self and other using the chain rule.

            Since d/dx (x * y) = y and d/dy (x * y) = x, we propagate:
                - other.value * out.grad to self.grad
                - self.value * out.grad to other.grad

            We use += because a variable might be used in multiple operations,
            meaning its gradient needs to accumulate from multiple contributions.
            """
            if self.requires_grad:
                self.grad += other.value * out.grad  # ∂L/∂self += ∂L/∂out * ∂out/∂self (which is other.value)
            if other.requires_grad:
                other.grad += self.value * out.grad  # ∂L/∂other += ∂L/∂out * ∂out/∂other (which is self.value)

        out._backward = _backward  # Store the backward function for this operation
        out._prev = {self, other}  # Track dependencies
        return out

    def backward(self):
        """
        Perform backpropagation to compute gradients of all dependent variables.
        """
        # Reset all gradients in the computation graph
        def reset_grads(node):
            if node.grad is not None:
                node.grad = 0
            for parent in node._prev:
                reset_grads(parent)

        reset_grads(self)  # Clear previous gradients
        self.grad = 1  # Seed gradient for the output (∂L/∂self = 1)

        topo = []  # Store nodes in topological order
        visited = set()

        def build_topo(node):
            """ Perform depth-first traversal to build a valid topological order """
            if node not in visited:
                visited.add(node)
                for parent in node._prev:
                    build_topo(parent)
                topo.append(node)

        build_topo(self)  # Construct topological order

        for node in reversed(topo):
            node._backward()  # Apply stored _backward function to propagate gradients

# Define a and b
a = DiffScalar(2, requires_grad=True)
b = DiffScalar(3, requires_grad=True)

# Compute e = (a + b) * (b + 1)
e = (a + b) * (b + 1)

# Perform backward pass
e.backward()

# Print gradients
print(a.grad)
print(b.grad)


4
9


Let's compare to pyTorch using the same example:

In [2]:
import torch

# Define scalars with requires_grad=True to track gradients
a = torch.tensor(2.0, requires_grad=True)  # Equivalent to DiffScalar(2, requires_grad=True)
b = torch.tensor(3.0, requires_grad=True)  # Equivalent to DiffScalar(1, requires_grad=True)

# Compute e = (a + b) * (b + 1)
e = (a + b) * (b + 1)  # PyTorch automatically tracks computation

# Perform backward pass
e.backward()  # Computes gradients

# Print gradients
print(a.grad)
print(b.grad)


tensor(4.)
tensor(9.)
